In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler

In [50]:
data = pd.read_csv("/home/mert/Desktop/MLOPS/bitcoin-forecast/data/raw/btc_usd.csv")
print(type(data.btc_vwap))
btc_vwap = data['btc_vwap']
btc_vwap = np.array(btc_vwap)
print(type(btc_vwap))
btc_vwap = btc_vwap.reshape(-1,1)

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


In [51]:
data['Datetime']   = pd.to_datetime(data['btc_timestamp']) #Replace 'time' with your actual datetime column name
data['year']       = data['Datetime'].dt.year
data['month']      = data['Datetime'].dt.month
data['dayofmonth'] = data['Datetime'].dt.day
data['dayofyear']  = data['Datetime'].dt.dayofyear
data['weekday']    = data['Datetime'].dt.weekday
data['date']       = data['Datetime'].dt.date
data['wntr_month'] = data['month'].apply(lambda x:1 if x in [11, 12, 1, 2] else 0)
data['month_label']= data['month'].map(lambda x:1 if x in [11, 12, 1, 2, 3] else (2 if x in [6, 7, 8, 9] else 3))
data['quarter']    = data['month'].map(lambda x:1 if x in [1, 2, 3] else (2 if x in [4, 5, 6] else (3 if x in [7, 8, 9] else 4)))
data['season']     = data['month'].map(lambda x:1 if x in [12, 1, 2] else (2 if x in [3, 4, 5] else (3 if x in [6, 7, 8] else 4)))
data['days']       = data['dayofmonth'].map(lambda x:1 if x < 5 else 0)
data['btc_vwap']   = (data['btc_vwap'] - data['btc_vwap'].mean()) / data['btc_vwap'].std()
data["rolling_30"] = data['btc_vwap'].rolling(window=30).mean()

lag_periods = [1, 15, 90]

for lag in lag_periods:
    data[f'btc_vwap_lag_{lag}'] = data['btc_vwap'].shift(lag)
    

# StandardScaler
scaler = StandardScaler()

for i in data.columns:
    print(type(data[i]))
    if type(data[i]) == "float64":
        scaler.fit(data[i])
        print('here')

for i in data.columns:
    data[i] = np.array(data[i]).reshape(-1, 1)
    data[i] = scaler.fit(data[i])
    print('here')

        
# Drop rows with NaN values (resulting from shifting)
data.dropna(inplace=True)
data = data.drop("btc_timestamp", axis=1)
data.set_index('Datetime', inplace=True)
data = data.sort_index()
data = data.iloc[:-1]


# Encode cyclical features
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col] / max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col] / max_val)
    return data

data = encode(data, 'dayofyear', 365)
data = encode(data, 'weekday', 52)
data = encode(data, 'month', 12)

data.to_csv("/home/mert/Desktop/MLOPS/bitcoin-forecast/data/external/btc_usd_features.csv", index=False)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


ValueError: Expected 2D array, got 1D array instead:
array=[29255.71   29678.34   33769.52   ... 46229.18   47357.4365 48278.585 ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
data

,btc_open,btc_high,btc_low,btc_close,btc_volume,btc_trade_count,btc_vwap,year,month,dayofmonth,...,rolling_30,btc_vwap_lag_1,btc_vwap_lag_15,btc_vwap_lag_90,dayofyear_sin,dayofyear_cos,weekday_sin,weekday_cos,month_sin,month_cos
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-04-01,58895.5000,60250.0000,57908.5400,59537.230,734.952428,39904.0,1.928417,2021,4,1,...,1.630662,1.903668,1.780514,-0.467210,0.999991,0.004304,0.354605,0.935016,0.866025,-0.5
2021-04-02,59549.2600,59719.3300,58440.3200,59401.970,469.544635,28460.0,1.945619,2021,4,2,...,1.653125,1.928417,1.863980,-0.255218,0.999917,-0.012910,0.464723,0.885456,0.866025,-0.5
2021-04-03,59401.9700,59796.5600,56382.4800,57579.190,765.320142,36330.0,1.848253,2021,4,3,...,1.679446,1.945619,1.897147,-0.128921,0.999546,-0.030120,0.568065,0.822984,0.866025,-0.5
2021-04-04,57575.4600,58508.0000,57058.8200,57310.410,431.529377,24448.0,1.836594,2021,4,4,...,1.705387,1.848253,1.899310,-0.309605,0.998880,-0.047321,0.663123,0.748511,0.866025,-0.5
2021-04-05,57310.2000,59499.0000,56819.1200,58807.070,618.376235,32945.0,1.891354,2021,4,5,...,1.732015,1.836594,1.768302,-0.157502,0.997917,-0.064508,0.000000,1.000000,0.866025,-0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-06,42733.2850,43359.9700,42616.5520,43000.361,0.865373,94.0,0.644949,2024,2,6,...,0.613939,0.625737,0.416567,0.024264,0.594727,0.803928,0.120537,0.992709,0.866025,0.5
2024-02-07,42990.7340,44774.8415,42822.2650,44547.857,4.170805,148.0,0.666514,2024,2,7,...,0.605522,0.644949,0.329273,0.147346,0.608477,0.793572,0.239316,0.970942,0.866025,0.5
2024-02-08,44541.7040,46390.1650,44437.9955,46263.479,2.405754,139.0,0.819259,2024,2,8,...,0.601741,0.666514,0.388192,0.164827,0.622047,0.782980,0.354605,0.935016,0.866025,0.5


In [ ]:
normalized_arr = (np.array([data['btc_vwap']])).reshape(-1,1)
# print(normalized_arr.reshape(-1,1))


In [ ]:
normalized_arr

array([[1.92841746],
       [1.94561944],
       [1.84825345],
       ...,
       [0.81925874],
       [0.98448612],
       [1.00469972]])